In [15]:
import os
import os.path
import sys
import pandas as pd
import numpy as np

In [16]:
LOCAL_PATH = 'data'
RAW_TRACE = 'ashakan_raw.pcapng'
TRACE_FEATURE_FILE = 'tcp_flow_features.csv'
BUCKET_NAME = 'edu.nyu.hsn.ddos-data' # replace with your bucket name
KEY = 'CAP_NIC1_00931_20130727230801.dms' # replace with your object key

In [17]:
if not os.path.exists(LOCAL_PATH + "/" + RAW_TRACE):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, LOCAL_PATH + "/" + RAW_TRACE)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [18]:
import subprocess
if not os.path.exists(LOCAL_PATH + "/" + TRACE_FEATURE_FILE):
    tshark_command = subprocess.Popen('tshark -r {} -Y tcp -T fields -e ip.src -e ip.dst -e tcp.srcport -e tcp.dstport -e tcp.len -e frame.time_relative -e tcp.seq -e tcp.ack -e tcp.flags.ack -e tcp.flags.syn -e tcp.flags.fin -e tcp.stream -Eheader=y -Eseparator=, > {}'.format(LOCAL_PATH + "/" + RAW_TRACE, LOCAL_PATH + "/" + TRACE_FEATURE_FILE), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out_data, err_data = tshark_command.communicate()
    out_data, err_data = out_data.decode('utf-8'), err_data.decode('utf-8')
    if err_data != '':
        pass
#         print(err_data)

In [19]:
trace_df = pd.read_csv(LOCAL_PATH + "/" + TRACE_FEATURE_FILE)
trace_df['src_addr'] = trace_df['ip.src'] + ":" + trace_df['tcp.srcport'].apply(str)
trace_df['dst_addr'] = trace_df['ip.dst'] + ":" + trace_df['tcp.dstport'].apply(str)



0.2 samp rate:   1%|          | 104/14419 [00:16<03:54, 61.08it/s]

In [20]:
trace_df.shape

(3275767, 14)

In [21]:
trace_df.dtypes

ip.src                  object
ip.dst                  object
tcp.srcport              int64
tcp.dstport              int64
tcp.len                  int64
frame.time_relative    float64
tcp.seq                  int64
tcp.ack                  int64
tcp.flags.ack            int64
tcp.flags.syn            int64
tcp.flags.fin            int64
tcp.stream               int64
src_addr                object
dst_addr                object
dtype: object

In [22]:
trace_df.describe()

,tcp.srcport,tcp.dstport,tcp.len,frame.time_relative,tcp.seq,tcp.ack,tcp.flags.ack,tcp.flags.syn,tcp.flags.fin,tcp.stream
count,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06,3.275767e+06
mean,1.959040e+04,1.189782e+04,1.001266e+03,2.770661e+01,3.896565e+08,5.583203e+07,9.967156e-01,6.052323e-03,5.241521e-03,1.130049e+03
std,2.130404e+04,1.709713e+04,6.454179e+02,1.189198e+01,4.620892e+08,1.584937e+08,5.721569e-02,7.756091e-02,7.220837e-02,3.101892e+03
min,2.100000e+01,2.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.450000e+02,2.770000e+03,1.180000e+02,1.734808e+01,8.033140e+05,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,2.770000e+03,2.901000e+03,1.460000e+03,2.784759e+01,2.364588e+08,3.034090e+05,1.000000e+00,0.000000e+00,0.000000e+00,1.400000e+01
75%,4.092000e+04,9.876000e+03,1.460000e+03,3.792557e+01,6.421602e+08,1.348768e+06,1.000000e+00,0.000000e+00,0.000000e+00,3.800000e+01
max,6.553400e+04,6.551600e+04,1.460000e+03,4.828968e+01,4.294967e+09,4.294967e+09,1.000000e+00,1.000000e+00,1.000000e+00,1.798300e+04


In [23]:
trace_df.head()

,ip.src,ip.dst,tcp.srcport,tcp.dstport,tcp.len,frame.time_relative,tcp.seq,tcp.ack,tcp.flags.ack,tcp.flags.syn,tcp.flags.fin,tcp.stream,src_addr,dst_addr
0,10.2.21.28,10.2.4.172,389,63633,293,0.000000,1,1,1,0,0,0,10.2.21.28:389,10.2.4.172:63633
1,10.2.4.179,10.2.4.146,40920,9876,1460,0.000001,1,1,1,0,0,1,10.2.4.179:40920,10.2.4.146:9876
2,10.2.4.179,10.2.4.146,40920,9876,1460,0.000002,1461,1,1,0,0,1,10.2.4.179:40920,10.2.4.146:9876
3,10.2.4.179,10.2.4.146,40920,9876,1460,0.000002,2921,1,1,0,0,1,10.2.4.179:40920,10.2.4.146:9876
4,10.2.4.179,10.2.4.146,40920,9876,1460,0.000002,4381,1,1,0,0,1,10.2.4.179:40920,10.2.4.146:9876


In [36]:
# run 'jupyter nbextension enable --py --sys-prefix widgetsnbextension' first
from tqdm import tqdm
def to_feature_df(raw_trace_df,sampling_rate=1.0):
    def calculate_two_way_tcp(df):
        def get_statistical_features(df, criter, feature_name,name_pred):
            feature_avg = df[criter][feature_name].mean()
            feature_avg = -1 if feature_avg is np.nan else feature_avg
            feature_min = df[criter][feature_name].min()
            feature_min = -1 if feature_min is np.nan else feature_min
            feature_max = df[criter][feature_name].max()
            feature_max = -1 if feature_max is np.nan else feature_max
            feature_std = df[criter][feature_name].std()
            feature_std = -1 if feature_std is np.nan else feature_std
            feature_sum = df[criter][feature_name].sum()
            feature_sum = -1 if feature_sum is np.nan else feature_sum
            feature_count = df[criter][feature_name].count()
            return {'avg('+name_pred+')':feature_avg,'std('+name_pred+')':feature_std,'min('+name_pred+')':feature_min,'max('+name_pred+')':feature_max,'count('+name_pred[0:8]+')':feature_count, 'sum('+name_pred+')':feature_sum}
        
        addrs = list(set(np.append(df['src_addr'].unique(), df['dst_addr'].unique())))
        if len(addrs) != 2:
            raise
        stat = get_statistical_features(df, df['src_addr'] == addrs[0],'tcp.len','forw_pkt_len')
        stat.update(get_statistical_features(df, df['src_addr'] == addrs[1],'tcp.len','back_pkt_len'))
        return pd.Series(stat)

    trace_df = raw_trace_df.sample(frac=sampling_rate)
    tcp_flow_df = pd.DataFrame()
    tcp_flow_df['avg(tcp_pkt_len)'] = trace_df.groupby('tcp.stream')['tcp.len'].mean()
    tcp_flow_df['stddev(tcp_pkt_len)'] = trace_df.groupby('tcp.stream')['tcp.len'].std().fillna(-1)
    tcp_flow_df['min(tcp_pkt_len)'] = trace_df.groupby('tcp.stream')['tcp.len'].min()
    tcp_flow_df['max(tcp_pkt_len)'] = trace_df.groupby('tcp.stream')['tcp.len'].max()
    tcp_flow_df['tot_pkt'] = trace_df.groupby('tcp.stream')['tcp.len'].count()
    tcp_flow_df['tot_byte'] = trace_df.groupby('tcp.stream')['tcp.len'].sum()
    tcp_flow_df['rel_start'] = trace_df.groupby('tcp.stream')['frame.time_relative'].min()
    tcp_flow_df['duration'] = trace_df.groupby('tcp.stream')['frame.time_relative'].max() - tcp_flow_df['rel_start']
    tqdm.pandas(desc='{} samp rate'.format(sampling_rate))
    two_way_flow_df = trace_df.groupby('tcp.stream')[['tcp.len','src_addr','dst_addr']].progress_apply(calculate_two_way_tcp)
    tcp_flow_df = pd.concat([tcp_flow_df,two_way_flow_df],axis=1)
    return tcp_flow_df

In [37]:
# run 'jupyter nbextension enable --py --sys-prefix widgetsnbextension' first
from tqdm import tqdm_notebook
# packet-based random sampling
for sampling_percent in tqdm_notebook([20,40,60,80,100],desc='Sampling'):
    sampling_rate = sampling_percent / 100.0
    to_feature_df(trace_df, sampling_rate).to_csv('{PATH}/packet_rand_{PERCENT}%.csv'.format(PATH=LOCAL_PATH,PERCENT=sampling_percent))

HBox(children=(IntProgress(value=0, description='Sampling', max=5), HTML(value='')))








0.2 samp rate:   0%|          | 0/14476 [00:00<?, ?it/s]






0.2 samp rate:   0%|          | 1/14476 [00:00<1:00:00,  4.02it/s]






0.2 samp rate:   0%|          | 4/14476 [00:00<54:02,  4.46it/s]  






0.2 samp rate:   0%|          | 14/14476 [00:00<38:38,  6.24it/s]






0.2 samp rate:   0%|          | 17/14476 [00:01<43:29,  5.54it/s]






0.2 samp rate:   0%|          | 24/14476 [00:01<31:33,  7.63it/s]






0.2 samp rate:   0%|          | 32/14476 [00:01<22:59, 10.47it/s]






0.2 samp rate:   0%|          | 39/14476 [00:01<17:10, 14.01it/s]






0.2 samp rate:   0%|          | 47/14476 [00:01<12:57, 18.56it/s]






0.2 samp rate:   0%|          | 57/14476 [00:02<09:47, 24.55it/s]






0.2 samp rate:   0%|          | 67/14476 [00:02<07:38, 31.45it/s]






0.2 samp rate:   1%|          | 75/14476 [00:02<06:28, 37.09it/s]






0.2 samp rate:   1%|          | 83/14476 [00:02<05:30, 43.58it/s]






0.2 samp rate:   1%|          | 92/14476 [00:02<04:41, 51.10it/s

0.2 samp rate:   5%|▌         | 776/14476 [00:10<02:49, 80.92it/s]






0.2 samp rate:   5%|▌         | 785/14476 [00:10<02:48, 81.18it/s]






0.2 samp rate:   5%|▌         | 794/14476 [00:11<02:44, 83.26it/s]






0.2 samp rate:   6%|▌         | 803/14476 [00:11<02:44, 82.96it/s]






0.2 samp rate:   6%|▌         | 812/14476 [00:11<02:46, 82.21it/s]






0.2 samp rate:   6%|▌         | 821/14476 [00:11<02:52, 79.19it/s]






0.2 samp rate:   6%|▌         | 830/14476 [00:11<02:48, 80.91it/s]






0.2 samp rate:   6%|▌         | 839/14476 [00:11<02:50, 80.12it/s]






0.2 samp rate:   6%|▌         | 848/14476 [00:11<02:47, 81.56it/s]






0.2 samp rate:   6%|▌         | 857/14476 [00:11<02:45, 82.14it/s]






0.2 samp rate:   6%|▌         | 866/14476 [00:11<02:44, 82.52it/s]






0.2 samp rate:   6%|▌         | 875/14476 [00:12<02:48, 80.62it/s]






0.2 samp rate:   6%|▌         | 884/14476 [00:12<02:44, 82.80it/s]






0.2 samp rate:   6%|▌         | 893/14476 [00:12<02

0.2 samp rate:  11%|█         | 1523/14476 [00:20<02:15, 95.27it/s]








                                                                  



0.2 samp rate:   1%|          | 158/14426 [03:49<5:45:33,  1.45s/it]






0.2 samp rate:  11%|█         | 1534/14476 [00:23<20:55, 10.31it/s]






0.2 samp rate:  11%|█         | 1546/14476 [00:23<15:10, 14.20it/s]






0.2 samp rate:  11%|█         | 1557/14476 [00:24<11:12, 19.22it/s]






0.2 samp rate:  11%|█         | 1567/14476 [00:24<08:35, 25.06it/s]






0.2 samp rate:  11%|█         | 1577/14476 [00:24<06:46, 31.75it/s]






0.2 samp rate:  11%|█         | 1586/14476 [00:24<05:27, 39.39it/s]






0.2 samp rate:  11%|█         | 1597/14476 [00:24<04:26, 48.27it/s]






0.2 samp rate:  11%|█         | 1609/14476 [00:24<03:39, 58.55it/s]






0.2 samp rate:  11%|█         | 1620/14476 [00:24<03:11, 67.20it/s]






0.2 samp rate:  11%|█▏        | 1631/14476 [00:24<03:00, 71.16it/s]






0.2 samp rate:  11%|█▏        | 1641/14

0.2 samp rate:  17%|█▋        | 2407/14476 [00:32<01:48, 111.33it/s]





0.2 samp rate:  17%|█▋        | 2421/14476 [00:32<01:41, 118.27it/s]





0.2 samp rate:  17%|█▋        | 2435/14476 [00:32<01:37, 123.06it/s]





0.2 samp rate:  17%|█▋        | 2448/14476 [00:32<01:39, 120.53it/s]





0.2 samp rate:  17%|█▋        | 2461/14476 [00:32<01:55, 104.36it/s]





0.2 samp rate:  17%|█▋        | 2472/14476 [00:33<01:53, 105.75it/s]





0.2 samp rate:  17%|█▋        | 2484/14476 [00:33<01:50, 108.19it/s]





0.2 samp rate:  17%|█▋        | 2496/14476 [00:33<01:51, 107.40it/s]





0.2 samp rate:  17%|█▋        | 2507/14476 [00:33<01:52, 106.69it/s]





0.2 samp rate:  17%|█▋        | 2519/14476 [00:33<01:48, 109.79it/s]





0.2 samp rate:  17%|█▋        | 2532/14476 [00:33<01:44, 113.82it/s]





0.2 samp rate:  18%|█▊        | 2545/14476 [00:33<01:40, 118.18it/s]





0.2 samp rate:  18%|█▊        | 2557/14476 [00:33<01:46, 111.98it/s]





0.2 samp rate:  18%|█▊        | 2569/1

...

1.0 samp rate:  72%|███████▏  | 12878/17985 [02:12<00:44, 115.09it/s]





1.0 samp rate:  72%|███████▏  | 12890/17985 [02:12<00:44, 115.76it/s]





1.0 samp rate:  72%|███████▏  | 12902/17985 [02:12<00:43, 115.75it/s]





1.0 samp rate:  72%|███████▏  | 12914/17985 [02:13<00:44, 115.21it/s]





1.0 samp rate:  72%|███████▏  | 12926/17985 [02:13<00:44, 114.89it/s]





1.0 samp rate:  72%|███████▏  | 12938/17985 [02:13<00:44, 112.94it/s]





1.0 samp rate:  72%|███████▏  | 12951/17985 [02:13<00:43, 116.07it/s]





1.0 samp rate:  72%|███████▏  | 12963/17985 [02:13<00:43, 116.17it/s]





1.0 samp rate:  72%|███████▏  | 12976/17985 [02:13<00:42, 117.13it/s]





1.0 samp rate:  72%|███████▏  | 12988/17985 [02:13<00:42, 117.36it/s]





1.0 samp rate:  72%|███████▏  | 13000/17985 [02:13<00:43, 114.85it/s]





1.0 samp rate:  72%|███████▏  | 13012/17985 [02:13<00:43, 115.33it/s]





1.0 samp rate:  72%|███████▏  | 13025/17985 [02:14<00:42, 117.52it/s]





1.0 samp rate:  72%|█████

1.0 samp rate:  78%|███████▊  | 13963/17985 [02:22<00:34, 116.05it/s]





1.0 samp rate:  78%|███████▊  | 13975/17985 [02:22<00:34, 116.16it/s]





1.0 samp rate:  78%|███████▊  | 13987/17985 [02:22<00:35, 112.61it/s]





1.0 samp rate:  78%|███████▊  | 14001/17985 [02:22<00:33, 118.18it/s]





1.0 samp rate:  78%|███████▊  | 14014/17985 [02:22<00:32, 121.08it/s]





1.0 samp rate:  78%|███████▊  | 14027/17985 [02:22<00:32, 121.80it/s]





1.0 samp rate:  78%|███████▊  | 14040/17985 [02:22<00:32, 119.96it/s]





1.0 samp rate:  78%|███████▊  | 14053/17985 [02:22<00:32, 120.04it/s]





1.0 samp rate:  78%|███████▊  | 14066/17985 [02:22<00:32, 119.55it/s]





1.0 samp rate:  78%|███████▊  | 14078/17985 [02:23<00:33, 117.80it/s]





1.0 samp rate:  78%|███████▊  | 14091/17985 [02:23<00:32, 119.35it/s]





1.0 samp rate:  78%|███████▊  | 14104/17985 [02:23<00:32, 120.98it/s]





1.0 samp rate:  78%|███████▊  | 14117/17985 [02:23<00:32, 118.57it/s]





1.0 samp rate:  79%|█████

1.0 samp rate:  84%|████████▍ | 15068/17985 [02:31<00:24, 118.73it/s]





1.0 samp rate:  84%|████████▍ | 15080/17985 [02:31<00:24, 117.61it/s]





1.0 samp rate:  84%|████████▍ | 15092/17985 [02:31<00:24, 116.71it/s]





1.0 samp rate:  84%|████████▍ | 15104/17985 [02:31<00:24, 115.71it/s]





1.0 samp rate:  84%|████████▍ | 15117/17985 [02:31<00:24, 118.02it/s]





1.0 samp rate:  84%|████████▍ | 15129/17985 [02:31<00:24, 116.31it/s]





1.0 samp rate:  84%|████████▍ | 15142/17985 [02:32<00:24, 118.26it/s]





1.0 samp rate:  84%|████████▍ | 15154/17985 [02:32<00:24, 117.30it/s]





1.0 samp rate:  84%|████████▍ | 15166/17985 [02:32<00:24, 114.62it/s]





1.0 samp rate:  84%|████████▍ | 15178/17985 [02:32<00:24, 114.69it/s]





1.0 samp rate:  84%|████████▍ | 15190/17985 [02:32<00:24, 114.99it/s]





1.0 samp rate:  85%|████████▍ | 15203/17985 [02:32<00:23, 117.37it/s]





1.0 samp rate:  85%|████████▍ | 15216/17985 [02:32<00:23, 118.95it/s]





1.0 samp rate:  85%|█████

1.0 samp rate:  90%|████████▉ | 16156/17985 [02:40<00:16, 113.77it/s]





1.0 samp rate:  90%|████████▉ | 16168/17985 [02:40<00:15, 114.89it/s]





1.0 samp rate:  90%|████████▉ | 16181/17985 [02:41<00:15, 116.25it/s]





1.0 samp rate:  90%|█████████ | 16193/17985 [02:41<00:15, 117.35it/s]





1.0 samp rate:  90%|█████████ | 16205/17985 [02:41<00:15, 116.08it/s]





1.0 samp rate:  90%|█████████ | 16218/17985 [02:41<00:14, 118.44it/s]





1.0 samp rate:  90%|█████████ | 16231/17985 [02:41<00:14, 119.92it/s]





1.0 samp rate:  90%|█████████ | 16244/17985 [02:41<00:14, 119.80it/s]





1.0 samp rate:  90%|█████████ | 16256/17985 [02:41<00:14, 119.77it/s]





1.0 samp rate:  90%|█████████ | 16268/17985 [02:41<00:14, 116.85it/s]





1.0 samp rate:  91%|█████████ | 16280/17985 [02:41<00:14, 116.30it/s]





1.0 samp rate:  91%|█████████ | 16292/17985 [02:42<00:14, 115.51it/s]





1.0 samp rate:  91%|█████████ | 16304/17985 [02:42<00:14, 115.85it/s]





1.0 samp rate:  91%|█████

1.0 samp rate:  96%|█████████▌| 17246/17985 [02:50<00:06, 117.89it/s]





1.0 samp rate:  96%|█████████▌| 17258/17985 [02:50<00:06, 115.88it/s]





1.0 samp rate:  96%|█████████▌| 17270/17985 [02:50<00:06, 115.06it/s]





1.0 samp rate:  96%|█████████▌| 17283/17985 [02:50<00:06, 116.39it/s]





1.0 samp rate:  96%|█████████▌| 17295/17985 [02:50<00:05, 116.31it/s]





1.0 samp rate:  96%|█████████▌| 17307/17985 [02:50<00:05, 115.94it/s]





1.0 samp rate:  96%|█████████▋| 17319/17985 [02:50<00:05, 112.48it/s]





1.0 samp rate:  96%|█████████▋| 17331/17985 [02:50<00:05, 113.40it/s]





1.0 samp rate:  96%|█████████▋| 17344/17985 [02:50<00:05, 115.81it/s]





1.0 samp rate:  97%|█████████▋| 17356/17985 [02:51<00:05, 115.17it/s]





1.0 samp rate:  97%|█████████▋| 17368/17985 [02:51<00:05, 114.70it/s]





1.0 samp rate:  97%|█████████▋| 17380/17985 [02:51<00:05, 115.93it/s]





1.0 samp rate:  97%|█████████▋| 17393/17985 [02:51<00:05, 117.96it/s]





1.0 samp rate:  97%|█████